In [1]:
%pip install "git+https://github.com/jemisjoky/TorchMPS.git"
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
%pip install torchmetrics
from torchmetrics.classification import MulticlassAccuracy
from torchmps import MPS

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/jemisjoky/TorchMPS.git to c:\users\piche\appdata\local\temp\pip-req-build-79zbyyc4
  Resolved https://github.com/jemisjoky/TorchMPS.git to commit 6c0bc1a8e2c15acba8570ca9ffe2b4a0c7135165
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/jemisjoky/TorchMPS.git 'C:\Users\piche\AppData\Local\Temp\pip-req-build-79zbyyc4'
c:\ProgramData\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


^C
Note: you may need to restart the kernel to use updated packages.


Defaulting to user installation because normal site-packages is not writeable


# Hyperparameters

In [ ]:
# Hardware hyperparameters
chosen_device = torch.device('cuda' 
if torch.cuda.is_available() else 'cpu')

# Data hyperparameters
nb_train_HP = 2
nb_test_HP = 500
batch_sz_HP = 150
batch_sz_HP = min(batch_sz_HP, nb_train_HP)
nb_classes_HP = 10

# Teacher hyperparameters
chosen_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
nteacher_epochs_HP = 15
hidden_size_HP = 70
teacher_loss_HP = nn.CrossEntropyLoss()
#Optimizer parameters
teacher_lr_HP = 0.0001
teacher_lr_HP = 0.9

# Student hyperparameters
# MPS parameters
bond_dim_HP = 20
adaptive_mode_HP = False
periodic_bc_HP = False
feature_map_HP = lambda x : torch.tensor([1, x]).to(chosen_device)
# Training parameters
nepochs_student_HP = 15
student_lr_HP = 0.0001
student_reg_HP = 0.0
student_loss_HP = nn.KLDivLoss(reduction="batchmean", log_target=True)
# Gaussian parameters
ngauss_epochs_HP = 5 # number of epochs with added gaussian noise
gn_var_HP = 0.3 #added gaussian noise variance
gn_mean_HP = 0 #added gaussian noise mean
# well suited loss for comparison of divergences

In [ ]:
# Import the mnist dataset
train_set = torchvision.datasets.MNIST(root = './datasets', train = True,   
    transform = transforms.ToTensor(),  download = True )

train_subset = torch.utils.data.SubsetRandomSampler(range(nb_train))

train_iterator = torch.utils.data.DataLoader(dataset = train_set, 
    sampler = train_subset, batch_size=batch_sz_HP)


test_set = torchvision.datasets.MNIST(root = './datasets',
 train = False, transform = transforms.ToTensor(),  download = True)

test_subset = torch.utils.data.SubsetRandomSampler(range(nb_test))

test_iterator = torch.utils.data.DataLoader(dataset = test_set, 
    sampler = test_subset, batch_size = batch_sz_HP)

# Training the base model

In [ ]:

# Create the fcnn class
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN, self).__init__()
        self.relu = nn.ReLU()
        self.lin1 = nn.Linear(784, hidden_size)
        self.lin2 = nn.Linear(hidden_size_HP, hidden_size_HP)
        self.lin3 = nn.Linear(hidden_size_HP, hidden_size_HP)
        self.lin4 = nn.Linear(hidden_size_HP, hidden_size_HP)
        self.lin5 = nn.Linear(hidden_size_HP, hidden_size_HP)
        self.lin6 = nn.Linear(hidden_size_HP, 10)

    def forward(self, x):
        y = self.lin1(x)
        y = self.relu(y)
        y = self.lin2(y)
        y = self.relu(y)
        y = self.lin3(y)
        y = self.relu(y)
        y = self.lin4(y)
        y = self.relu(y)
        y = self.lin5(y)
        y = self.relu(y)
        y = self.lin6(y)
        y = self.relu(y)
        return y

#Instantiate and put the model on the chosen device
teacher = FCNN().to(chosen_device)

#Instantiate the optimizer
teacher_optimizer = torch.optim.Adam(teacher.parameters())

#Training loop
for epoch in range(n_epochs_fcnn):
    for (x_mb, y_mb) in train_iterator:
        # Reshape the train_tuple and put on the chosen device
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)
        # Foward propagation
        y_hat_mb = teacher(x_mb)
        # Backpropagation
        teacher_loss_HP(y_hat_mb, y_mb).backward()
        teacher_optimizer.step()
        teacher_optimizer.zero_grad()

    print(loss.item())

# Get the validation set classification accuracy
teacher_acc_score = 0
teacher_acc_metric = MulticlassAccuracy(num_classes=nb_classes_HP).to(chosen_device)
for (x_mb, y_mb) in test_iterator:
    x_mb = x_mb.reshape(-1, 784).to(chosen_device)
    y_mb = y_mb.to(chosen_device)
    # add the number of datapoints we classified right
    teacher_acc_score += x_mb.size()[0] * teacher_acc_metric( teacher(x_mb), y_mb )
print("The teacher's accuracy score is:")
print(teacher_acc_score / nb_test) #divide by total size



2.3011550903320312
2.2921862602233887
2.2846620082855225
2.277676582336426
2.270646572113037
2.2635183334350586
2.2557387351989746
2.2475104331970215
2.23850154876709
2.2281408309936523
2.21614933013916
2.201730728149414
2.185075283050537
2.1674695014953613
2.147073268890381
The teacher's accuracy score is:
tensor(0.1000)


# Training the student model

In [ ]:

# Initialize the MPS module
student = MPS(
    input_dim = 28 ** 2,
    output_dim = 10,
    bond_dim = bond_dim_HP,
    adaptive_mode = adaptive_mode_HP,
    periodic_bc = periodic_bc_HP,
).to(chosen_device)
student.register_feature_map(feature_map_HP)

# Instantiate the optimizer and softmax
student_optimizer = torch.optim.Adam(
    student.parameters(), lr = student_lr_HP, weight_decay = student_reg_HP
    )

# Used on the inputs before the loss function
LogSoftmax = nn.LogSoftmax(dim=1)

# Training loop 
for epoch in range(nepochs_student_HP):
    # Create an array to store the val loss
    # of the student at each epoch
    stud_val_loss = []
    for (x_mb, _) in train_iterator:
        
        # Flatten the MNIST images, which come in matrix form
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)

        # Apply Gaussian noise for the gaussian noised epochs
        if epoch > (nepochs_student_HP - ngauss_epochs_HP):
          x_mb = (x_mb + torch.randn(size=x_mb.size())).to(chosen_device)

        # Get log of softmax of the teacher and student logits
        # required before passing to KL divergence loss in Pytorch for some reason
        teacher_output = LogSoftmax(teacher(x_mb)).to(chosen_device)
        student_output = LogSoftmax(student(x_mb)).to(chosen_device)

        # Backpropagation
        student_loss_HP(student_output, teacher_output).backward()
        student_optimizer.step()
        student_optimizer.zero_grad()

    print(loss.item())



# Returns the validation set classification accuracy
# test set accuracy of the student 
def get_student_val_acc():
    # Get the validation set classification accuracy
    student_acc_score = 0
    student_acc_metric = MulticlassAccuracy(num_classes=nb_classes).to(chosen_device)
    for (x_mb, y_mb) in test_iterator:
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)
        # add the number of datapoints we classified right
        student_acc_score += x_mb.size()[0] * student_acc_metric( student(x_mb), y_mb )
    print("Student_acc_score:")
    print(student_acc_score / nb_test) #divide by total size


2.3025853633880615
2.3025758266448975
2.3025600910186768
2.3024463653564453
2.3020477294921875
2.299933433532715
2.2955358028411865
2.2959940433502197
2.2951107025146484
2.2976527214050293
2.296163558959961
2.297980546951294
2.298276662826538
2.29848051071167
2.2979133129119873
Student_acc_score:
tensor(0.1000)
